In [1]:
import numpy as np
import cv2
import time
import os
import glob
import math
import datetime
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib import path
import pickle
from scipy import ndimage

from scipy.ndimage.morphology import grey_dilation, grey_closing, grey_opening
from scipy.ndimage import generate_binary_structure, gaussian_filter

# print opencv version
ver = cv2.__version__
print('---OpenCV Version: ' + ver)

Using matplotlib backend: Qt5Agg
---OpenCV Version: 3.4.0


In [2]:
%qtconsole

## Define Functions

In [3]:
def load_video(file):

    cap = cv2.VideoCapture(file)
    width = int(cap.get(3))
    height = int(cap.get(4))
    vid_length = np.min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),500])

    # preallocate
    frames_raw = np.zeros((vid_length, height, width), np.uint8)

    rgb_convert = np.array([[[0.2989]],[[0.5870]],[[0.1140]]]).T

    for kk in range(vid_length):
        # Capture frame-by-frame
        ret, frame2 = cap.read()
        if ret:
            img = np.sum(frame2*rgb_convert, axis = 2)
            frames_raw[kk,:,:] = img
        else:
            print('skipping fr %i'%kk)

    cap.release()
    print('-- %i video frames have been loaded!'%vid_length)
    return width, height, vid_length, frames_raw


In [4]:
def check_rectangles(file):
    file_root = file.split('/')[-1]
    sname = file[:-4] + '_TunnelCorners.pkl'
    if os.path.isfile(sname):
        with open(sname, 'rb') as f:
            rectangle_corners = pickle.load(f)
        if rectangle_corners[0] is None:
            print('%s -- None'%file_root)
    else:
        print('%s -- no file'%file_root)
    return

def get_rectangles(file):
    sname = file[:-4] + '_TunnelCorners.pkl'
    if os.path.isfile(sname):
        with open(sname, 'rb') as f:
            rectangle_corners = pickle.load(f)
        if not rectangle_corners[0] is None :
            print('loaded rectangle corners from file')
            return rectangle_corners
        else:
            print('problem with rectangles file')
            return np.nan
    else:
        print('no rectangle file!')
        return np.nan
    
# Calculate mask
def calculate_mask(rectangle_corners, height, width):

    mask = np.zeros((height,width,6))
    tmp = np.full((height,width),True)
    idcs = np.where(tmp)
    for rect_num in range(0,6):
        p = path.Path(rectangle_corners[rect_num], closed=False)
        tmp2 = p.contains_points(list(zip(*reversed(idcs))), radius = -1 )
        mask[idcs[0][tmp2],idcs[1][tmp2],rect_num]=1
    del tmp, tmp2, idcs, p

    return mask

In [5]:
def even_illumination(img, SE_size):
    SE = np.ones((SE_size,SE_size)).astype(bool)
    img_clos = grey_closing(img, structure = SE)
#     img_open = grey_opening(img, structure = SE)
    img_bkg = img/img_clos
#     img_bkg = 3*img-img_open-img_clos
    return img_bkg

def get_rid_of_blobs(bkg_div_mask):
    cleaned_bkg_div = bkg_div_mask.copy()
    for FrNum in range(bkg_div_mask.shape[0]):
        frame =bkg_div_mask[FrNum,:,:]>1.15
        blobs, number_of_blobs = ndimage.label(frame)
        loc = ndimage.find_objects(blobs)
        means = np.full((number_of_blobs,2),np.nan)
        n_pts = np.full((number_of_blobs),np.nan)
        for kk in range(number_of_blobs):
            temp = np.where(blobs==kk+1)
            if len(temp[0])>6 and len(temp[0])<100:
                means[kk,:] = [np.mean(temp[1]), np.mean(temp[0])]
                n_pts[kk] = len(temp[0])
        x_dist = np.abs(means[:,0][:,None]- means[:,0][None,:])
        y_dist = np.abs(means[:,1][:,None]- means[:,1][None,:])
        total_dist = np.sqrt(np.square(x_dist)+np.square(y_dist))
        close_blobs, counts = np.unique(np.where(np.logical_and(total_dist<30,total_dist>0.001))[0], return_counts=True)
        blobs_to_remove = np.concatenate((close_blobs[counts>3], np.where(np.isnan(means[:,0]))[0]))
        blobs_to_keep = np.delete(np.arange(0,number_of_blobs),blobs_to_remove)
        where_in_frame_remove = np.isin(blobs,blobs_to_remove+1)
        cleaned_bkg_div[FrNum,where_in_frame_remove]=0
    return cleaned_bkg_div


def find_ant_pix_allframes(frames_raw, buffer, mask, bkg_cutoff, height, width, rectangle_corners):
    frames = 255-frames_raw
    del frames_raw
    
    # combine all masks
    mask_all = np.sum(mask,axis=2).astype(bool)

     # find places where is light throughout all of frames and dim
    print('-- removing light reflections')
    frames_even = frames.copy()
    n_brightframes = np.sum(frames[:-1,:,:]<100, axis =0)/frames.shape[0]*mask_all
    n_darkframes = np.sum(frames[:-1,:,:]>150, axis =0)/frames.shape[0]*mask_all
    frames_even[:,n_brightframes>0.7]=frames_even[:,n_brightframes>0.7]+50
    frames_even[:,n_darkframes>0.7]=frames_even[:,n_darkframes>0.7]-50
    
    # use before and after frames to get rid of background
    print('-- taking backgrond division using %i frames one each side of every frame'%buffer)
    ainsert = np.full((buffer, height, width), np.nan)
    tmp = frames_even.copy().astype(np.float)
    tmp = np.insert(tmp, 0, ainsert, axis = 0)
    tmp = np.append(tmp, ainsert, axis = 0)
    bkg = np.full(frames.shape, np.nan)
    for ii,f in enumerate(range(buffer,frames.shape[0]+buffer)):
        frs_OI = f+np.delete(np.arange(-1*buffer,buffer+1),buffer)
        bkg[ii,:,:]=np.nanmean(tmp[frs_OI,:,:], axis = 0)
    del tmp, ainsert
    bkg_div =np.divide(frames_even,bkg)
    bkg_div_mask = bkg_div.copy()
    bkg_div_mask[:,~mask_all]=0 #np.min(bkg_div[mask_all])
    del bkg_div, frames_even
    
    # get rid of small and large blobs
    print('-- getting rid of small/large blobs')
    bkg_div_blobs = get_rid_of_blobs(bkg_div_mask)
    
    
    # how many ant pixels
    print('-- finding ants in each tunnel')
    ant_pix = [np.where(bkg_div_blobs[f,:,:]>bkg_cutoff) for f in range(vid_length)]
    tunnel_ant_pix = [[None for x in range(6)] for y in range(vid_length)]
    tunnel_n_ant_pix = np.full((6,vid_length), np.nan)
    for rect_num in range(len(rectangle_corners)):
        for fr in range(frames.shape[0]):
            p = path.Path(rectangle_corners[rect_num], closed=False)
            if np.any(ant_pix[fr][0]) and ant_pix[fr][0].shape[0]<2000:
                tmp = p.contains_points(list(zip(*reversed(ant_pix[fr]))), radius = -1 )
                if np.sum(tmp)<1500:
                    tunnel_ant_pix[fr][rect_num] = np.array(ant_pix[fr])[:,tmp]
                    tunnel_n_ant_pix[rect_num, fr] = np.sum(tmp)
                    
                    
    
    
    del tmp, fr, mask_all, bkg_div_blobs
    return bkg_div_mask, ant_pix, tunnel_ant_pix, tunnel_n_ant_pix
    




In [6]:

def save_image(vlocation, nfig, name_base):
    pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
    plt.savefig(pname)
    nfig = nfig + 1
#     plt.pause(0.2)
#     plt.close('all')
    return nfig


def save_video(vlocation, name_base):
    # save images as movie
    if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
        os.remove(vlocation + "/%s.mp4"%name_base)
        print('-- Deleted %s.mp4 file'%name_base)
    print('saving %s.mp4 file'%name_base)
    command_p1 = "ffmpeg -r 20 -i '%s/%s"%(vlocation, name_base)
    command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
    command = command_p1 + "%01d.png'" + command_p2
#     print(command)
    os.system(command)
    plt.pause(10)
    print('-- Saved video file')
    
    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
    for pic in pics2delete:
        os.remove(pic)
    print('-- Deleted pics')
    return

## MAIN BODY OF PROGRAM

In [7]:
# where we lookin?
vfolder = '/media/gravishlab/SeagateExpansionDrive/AntTunnelPreference/Outdoor'
file_list = sorted(glob.glob(os.path.join(vfolder, '**[B,F].MP4')))

In [8]:
# which files do you need to select rectangles for?
for file in file_list:
    check_rectangles(file)
print('all done!')

all done!


In [9]:

vfolder = '/media/gravishlab/SeagateExpansionDrive/AntTunnelPreference/Outdoor'
file_list = sorted(glob.glob(os.path.join(vfolder, '**[B,F].MP4')))

all_tracked_data = [0]*len(file_list)

for ff,file in enumerate(file_list):
    sname = file[:-4] + '_TrackedData.pkl'
    
    if os.path.isfile(sname):
        print('loading tracked data from: %s'%file.split('/')[-1])
        with open(sname, 'rb') as f:
            all_tracked_data[ff] = pickle.load(f)
        f.close()
        
    else:

        # load video
        print('loading video frames from: ')
        print(file)
        width, height, vid_length, frames_raw = load_video(file)

        # load rectangles of interest to analyze
        rectangle_corners = get_rectangles(file)

        # calculate mask
        print('\ncalculating mask')
        mask = calculate_mask(rectangle_corners, height, width)

        # analyze video
        print('\nAnalyzing whole video to identify ants in each tunnel')
        buffer = 2
        bkg_div_mask, ant_pix, tunnel_ant_pix, tunnel_n_ant_pix = find_ant_pix_allframes(frames_raw, buffer, mask, 1.15, height, width, rectangle_corners)
        # tunnel_n_ant_pix[tunnel_n_ant_pix>1500]=np.nan
        
        all_tracked_data[ff] = tunnel_n_ant_pix

        print('\nSaving tracked data\n\n')
        sname = file[:-4] + '_TrackedData.pkl'
        with open(sname, 'wb') as f:
            pickle.dump(tunnel_n_ant_pix, f)
        f.close()

    
print('Done!')

loading tracked data from: 2018_10_25_B.MP4
loading tracked data from: 2018_10_25_F.MP4
loading tracked data from: 2018_10_26_B.MP4
loading tracked data from: 2018_10_26_F.MP4
loading tracked data from: 2018_10_29_F.MP4
loading tracked data from: 2018_10_30_B.MP4
loading tracked data from: 2018_10_30_F.MP4
loading tracked data from: 2018_10_31_B.MP4
loading tracked data from: 2018_10_31_F.MP4
loading tracked data from: 2018_11_2_B.MP4
loading tracked data from: 2018_11_2_F.MP4
loading tracked data from: 2018_11_3_B.MP4
loading tracked data from: 2018_11_3_F.MP4
loading tracked data from: 2018_11_4_F.MP4
loading tracked data from: 2018_11_5_F.MP4
Done!


In [10]:
# plot all data together
plt.close('all')
pltcolors = ['#BA4246', '#087E8B', '#701C6F']
plt.figure()
for ii,idc in enumerate(range(0,len(all_tracked_data))):
    
    if not isinstance(all_tracked_data[idc], int):


        total_ant_pix = np.nansum(all_tracked_data[idc], axis = 1)
        x_pos = np.repeat(np.arange(ii, ii+2*(len(all_tracked_data)+1)+1,len(all_tracked_data)+1),2)
        plt.bar(x_pos, total_ant_pix*np.array([1,-1,1,-1,1,-1]),align='edge', width = 1, 
             alpha = 0.4, linewidth = 0, color = np.repeat(pltcolors,2))
        plt.xticks( len(all_tracked_data)/2 + (len(all_tracked_data)+1)*np.array([0,1,2]), ('1 mm','3 mm', '5 mm'))
        plt.yticks([])
        plt.axhline(y=0, xmin=0, xmax=1,  alpha = 0.2, color = 'k', linewidth = 0.5)

plt.title('all trials', loc='left')
down, up = plt.ylim()
plt.text(-.5,up-1500, 'smooth')
plt.text(-.5,down+1000, 'rough')
plt.ylabel('total ant pixels')


# plot as percentage of total pixels
from scipy import stats
flat_and_rough_pixels = np.array(
    [np.sum(np.reshape(np.nansum(tdata,axis =1), (3,2)), axis =1) for tdata in all_tracked_data])
flat_pixels = np.array(
    [np.nansum(tdata[::2],axis =1) for tdata in all_tracked_data])
percent_flat_pixels = flat_pixels/flat_and_rough_pixels
cutoff = 5000 # at least 5000 total ant pixels to count trial
percent_flat_pixels[flat_and_rough_pixels < cutoff] = np.nan
print(percent_flat_pixels)
data_for_anova =  [[ g for g in gs if not math.isnan(g)] for gs in percent_flat_pixels.T.tolist()]
stats_2way = [stats.ttest_1samp(data, 0.5)[1] for data in data_for_anova]
stats_comparisons = []
comp_list = [0,1,2,0]
for cc in range(0,3):
    p_val = stats.ttest_ind(data_for_anova[comp_list[cc]], data_for_anova[comp_list[cc+1]])
    print(p_val)
    stats_comparisons.append(p_val)


plt.figure()
plt.bar(np.arange(1,6,2), np.nanmean(percent_flat_pixels, axis = 0), 
        yerr = np.nanstd(percent_flat_pixels, axis = 0), color = pltcolors)
for pp, p_val in enumerate(stats_2way):
    plt.text(1+2*pp, 0.9, 'p = %0.4f'%p_val, horizontalalignment = 'center')
    plt.text(2+2*pp, 0.3, 'p = %0.3f'%stats_comparisons[pp][1], horizontalalignment = 'center' )
plt.axhline(y=0.5, xmin=0, xmax=1,  alpha = 0.2, color = 'k', linewidth = 0.5)
plt.ylim((0,1))

[[0.84787076 0.86978373 0.84192726]
 [0.81239689 0.84883366 0.17485737]
 [0.78300631 0.36954853 0.69800789]
 [0.6060267  0.83540473 0.76599467]
 [0.31138806 0.71533129 0.39457976]
 [       nan 0.54734428 0.4020767 ]
 [0.7654937  0.81373546 0.75475294]
 [0.88999359 0.8393372  0.82728381]
 [0.6046625  0.71665494 0.74929563]
 [       nan        nan 0.91278218]
 [0.63292608 0.5665802  0.59571111]
 [0.81071994 0.78892006 0.72517452]
 [0.7147111  0.76234355 0.43563433]
 [0.72756654 0.18650023 0.25326547]
 [0.74180188 0.54603516 0.49326326]]
Ttest_indResult(statistic=0.5707214429098816, pvalue=0.5732832033641659)
Ttest_indResult(statistic=0.8720869382173182, pvalue=0.3908503086409232)
Ttest_indResult(statistic=-1.4800623011953986, pvalue=0.15087302948011136)


(0, 1)

In [53]:
# plot data by backyard or field
backyard_idcs = np.where(['B' in f for f in file_list])[0]
field_idcs = np.where(['F' in f for f in file_list])[0]

plt.close('all')
plt.figure()
for ii,idc in enumerate(backyard_idcs):
    
    if not isinstance(all_tracked_data[idc], int):


        total_ant_pix = np.nansum(all_tracked_data[idc], axis = 1)
        x_pos = np.repeat(np.arange(ii, ii+2*(len(backyard_idcs)+1)+1,len(backyard_idcs)+1),2)
        print(x_pos)
        plt.bar(x_pos, total_ant_pix*np.array([1,-1,1,-1,1,-1]),align='edge', width = 1, 
             alpha = 0.4, linewidth = 0, color = np.repeat(['r','g','b'],2))
        plt.xticks( len(backyard_idcs)/2 + (len(backyard_idcs)+1)*np.array([0,1,2]), ('1 mm','3 mm', '5 mm'))
        plt.yticks([])
        plt.axhline(y=0, xmin=0, xmax=1,  alpha = 0.2, color = 'k', linewidth = 0.5)

plt.title('backyard trials')
down, up = plt.ylim()
plt.text(-.5,up-1000, 'smooth')
plt.text(-.5,down+1000, 'rough')
plt.ylabel('total ant pixels')



    
# # #     for rect_num in range(0,6):
# # #         plt.plot(range(0,FrNum+1), np.nancumsum(tunnel_ant_pix[rect_num,:])[0:FrNum+1], 
# # #                          '-', color = pltcolors[rect_num], alpha = 0.9)
#     for rect_num in range(0,6):
#         plt.bar(range(0,FrNum+1)+np.floor(rect_num/2)/3, (rect_num%2*2-1)*tunnel_n_ant_pix[rect_num,:FrNum+1], 
#                          width=1, align='edge', color = pltcolors[rect_num], alpha = 0.3, linewidth = 0)
#         plt.axhline(y=0, xmin=0, xmax=1,  alpha = 0.2, color = 'k', linewidth = 0.5)
#     plt.xlim(0, vid_length)
#     plt.ylim(-1300, 1300)
#     plt.xlabel('Frame')
#     ax3.axes.get_yaxis().set_ticks([])
#     plt.ylabel('n ant pixels')
#     plt.text(vid_length-10, 1250, 'rough substrate', horizontalalignment='right', verticalalignment = 'top')
#     plt.text(vid_length-10, -1250, 'smooth substrate', horizontalalignment='right', verticalalignment = 'bottom')
    

[ 0  0  7  7 14 14]
[ 1  1  8  8 15 15]
[ 2  2  9  9 16 16]
[ 3  3 10 10 17 17]
[ 4  4 11 11 18 18]
[ 5  5 12 12 19 19]
[[0.84787076 0.86978373 0.84192726]
 [0.81239689 0.84883366 0.17485737]
 [0.78300631 0.36954853 0.69800789]
 [0.6060267  0.83540473 0.76599467]
 [0.31138806 0.71533129 0.39457976]
 [       nan 0.54734428 0.4020767 ]
 [0.7654937  0.81373546 0.75475294]
 [0.88999359 0.8393372  0.82728381]
 [0.6046625  0.71665494 0.74929563]
 [       nan        nan 0.91278218]
 [0.63292608 0.5665802  0.59571111]
 [0.81071994 0.78892006 0.72517452]
 [0.7147111  0.76234355 0.43563433]]


<BarContainer object of 3 artists>

## FOR A GIVEN FILE, SELECT RECTANGLES AND SAVE

In [20]:
# GUI TO FIND INDICES OF POLYGON FOR EACH RECTANGLE
file=file_list[-1]
print(file)
print('loading frames\n')
width, height, vid_length, frames_raw = load_video(file)

sname = file[:-4] + '_TunnelCorners.pkl'
global rect_num, rectangle_corners



# if os.path.isfile(sname):
#     print('loading rectangle corners from file')
#     with open(sname, 'rb') as f:
#         rectangle_corners = pickle.load(f)
# else:
#     print('no file')
from matplotlib.widgets import PolygonSelector

def line_select_callback(eclick, erelease):
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    print("(%3.2f, %3.2f) --> (%3.2f, %3.2f)" % (x1, y1, x2, y2))
    print(" The button you used were: %s %s" % (eclick.button, erelease.button))

def onselect(verts):
#         print(verts, rect_num)
        rectangle_corners[rect_num] = verts
#         self.ind = np.nonzero(path.contains_points(self.xys))[0]
#         self.fc[:, -1] = 0.3
# #         self.fc[self.ind, -1] = 1
#         self.collection.set_facecolors(self.fc)
#         self.canvas.draw_idle()

def toggle_selector(event):
    global rect_num
    if event.key=='escape' and toggle_selector.RS.active:
        print(' - Done!')
        rect_num = rect_num + 1
        if rect_num <= len(substrates)-1:
            print('--Select rectangle for tunnel: %i mm'%substrates[rect_num], end = '', flush =True)
        else:
            event.key = 'Q'
    if event.key in ['Q', 'q'] and toggle_selector.RS.active:
        print('\n RectangleSelector deactivated.')
        toggle_selector.RS.set_active(False)
    if event.key in ['A', 'a'] and not toggle_selector.RS.active:
        print('\n RectangleSelector activated.')
        toggle_selector.RS.set_active(True)



#     global rect_num, rectangle_corners
rect_num = 0
rectangle_corners = 6*[None]
substrates = [0,1,0,3,0,5]

plt.close('all')
fig = plt.figure(figsize = (18,10))
plt.title('Select rectangles for each tunnel')
current_ax = plt.gca()
plt.imshow(frames_raw[30,:,:], cmap = 'gray')
print('Select rangles for each tunnel!')
print('ESC to go to next tunnel')
print('Q or q to quit')
print('Select corners in clockwise direction')
print('\n--Select rectangle for tunnel: %i mm'%substrates[rect_num], end = '', flush =True)
# toggle_selector.RS = RectangleSelector(current_ax, line_select_callback,
#                                        drawtype='box', useblit=True,
#                                        button=[1, 3],  # don't use middle button
#                                        minspanx=5, minspany=5,
#                                        spancoords='pixels',
#                                        interactive=True)

toggle_selector.RS = PolygonSelector(current_ax, onselect, 
                                     useblit=False, 
                                     lineprops=None, 
                                     markerprops=None, 
                                     vertex_select_radius=15)
plt.connect('key_press_event', toggle_selector)
plt.show()
    
# if not os.path.isfile(sname):
#     print('saving rectangle corners to file')
#     with open(sname, 'wb') as f:
#         pickle.dump(rectangle_corners, f)

/media/gravishlab/SeagateExpansionDrive/AntTunnelPreference/Outdoor/2018_11_5_F.MP4
loading frames

-- 500 video frames have been loaded!
Select rangles for each tunnel!
ESC to go to next tunnel
Q or q to quit
Select corners in clockwise direction

--Select rectangle for tunnel: 0 mm - Done!
--Select rectangle for tunnel: 1 mm - Done!
--Select rectangle for tunnel: 0 mm - Done!
--Select rectangle for tunnel: 3 mm - Done!
--Select rectangle for tunnel: 0 mm - Done!
--Select rectangle for tunnel: 5 mm - Done!

 RectangleSelector deactivated.


In [21]:

def save_rectangles(file, rectangle_corners):
    sname = file[:-4] + '_TunnelCorners.pkl'
    if os.path.isfile(sname):
        os.remove(sname)
    
    print('saving rectangle corners to file')
    with open(sname, 'wb') as f:
        pickle.dump(rectangle_corners, f)
    return

save_rectangles(file, rectangle_corners)

saving rectangle corners to file


## Plot and save analysis

In [19]:
save_vid = True

file = file_list[-1]

# load video
print('loading video frames from: ')
print(file)
width, height, vid_length, frames_raw = load_video(file)

# load rectangles of interest to analyze
rectangle_corners = get_rectangles(file)

# calculate mask
print('\ncalculating mask')
mask = calculate_mask(rectangle_corners, height, width)

# analyze video
print('\nAnalyzing whole video to identify ants in each tunnel')
buffer = 2
bkg_div_mask, ant_pix, tunnel_ant_pix, tunnel_n_ant_pix = find_ant_pix_allframes(frames_raw, buffer, mask, 1.15, height, width, rectangle_corners)


substrates = [0,1,0,3,0,5]
pltcolors =['r','r','g','g','b','b']
pltlinestyles = ['--','-']
plt.close('all')

plt.figure(figsize = (18,5))
for FrNum in range(0,vid_length):
    plt.clf()

# #     # use image processing to find ant_pix
# #     ant_pix, img_bkg, total_ant_pix = find_ant_pix(frames, FrNum, buffer, mask, 1.15)


    ax1 = plt.subplot(1,3,1)
    plt.imshow(frames_raw[FrNum,:,:],cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=255))
    plt.text(10,-30, 'Fr: %i'%FrNum, color = [0.2,0.2,0.2])
    plt.text(10,-100, 'File: %s'%file.split('/')[-1], color = [0.2,0.2,0.2])
    ax2=plt.subplot(1,3,2)
    plt.imshow(bkg_div_mask[FrNum,:,:], cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=1.5))
    
    for rect_num in range(0,6):
        for ax in {ax1,ax2}:
            plt.sca(ax)
            patch = patches.PathPatch(path.Path(rectangle_corners[rect_num], closed=False), 
                                  facecolor='none', lw=0.5,
                                 ec = pltcolors[rect_num], ls = pltlinestyles[rect_num%2])
            p = path.Path(rectangle_corners[rect_num], closed=False)
            plt.gca().add_patch(patch)
            if (rect_num%2==1) and (ax==ax1):
                plt.text(10, 50+rect_num*20, '%i mm'%substrates[rect_num], color = pltcolors[rect_num])

            if tunnel_n_ant_pix[rect_num,FrNum]>0 and np.sum(tunnel_n_ant_pix[:,FrNum])<2000:
                plt.plot(tunnel_ant_pix[FrNum][rect_num][1], tunnel_ant_pix[FrNum][rect_num][0], 
                         '.', color = pltcolors[rect_num], MarkerSize = 0.5, alpha = 0.5)
#                 else:
#                     print('too many ant points = likely lighting change %i'%FrNum)
    for ax in {ax1,ax2}:
        plt.sca(ax)
        plt.axis('off')
    # restict axis 2 to be just tunnels
    mins=np.min(np.min(np.array(rectangle_corners),axis =1),axis=0)
    maxs=np.max(np.max(np.array(rectangle_corners),axis =1),axis=0)
    ax2.set_xlim([mins[0]-5,maxs[0]+5])
    ax2.set_ylim([maxs[1]+5,mins[1]-5])
        
        
    ax3 = plt.subplot(1,3,3)
# #     for rect_num in range(0,6):
# #         plt.plot(range(0,FrNum+1), np.nancumsum(tunnel_ant_pix[rect_num,:])[0:FrNum+1], 
# #                          '-', color = pltcolors[rect_num], alpha = 0.9)
    for rect_num in range(0,6):
        plt.bar(range(0,FrNum+1)+np.floor(rect_num/2)/3, (rect_num%2*2-1)*tunnel_n_ant_pix[rect_num,:FrNum+1], 
                         width=1, align='edge', color = pltcolors[rect_num], alpha = 0.3, linewidth = 0)
        plt.axhline(y=0, xmin=0, xmax=1,  alpha = 0.2, color = 'k', linewidth = 0.5)
    plt.xlim(0, vid_length)
    plt.ylim(-1300, 1300)
    plt.xlabel('Frame')
    ax3.axes.get_yaxis().set_ticks([])
    plt.ylabel('n ant pixels')
    plt.text(vid_length-10, 1250, 'rough substrate', horizontalalignment='right', verticalalignment = 'top')
    plt.text(vid_length-10, -1250, 'smooth substrate', horizontalalignment='right', verticalalignment = 'bottom')
    
    

    plt.pause(0.5)
    if save_vid:
        save_image(vfolder, FrNum, file.split('/')[-1][:-4]+'_Analysis')
    
if save_vid:
    save_video(vfolder, file.split('/')[-1][:-4]+'_Analysis')
    
    
print('\n\nDone!')

loading video frames from: 
/media/gravishlab/SeagateExpansionDrive/AntTunnelPreference/Outdoor/2018_10_27_F.MP4
-- 484 video frames have been loaded!
loaded rectangle corners from file

calculating mask

Analyzing whole video to identify ants in each tunnel
-- removing light reflections
-- taking backgrond division using 2 frames one each side of every frame
-- getting rid of small/large blobs


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater


-- finding ants in each tunnel
saving 2018_10_27_F_Analysis.mp4 file
-- Saved video file
-- Deleted pics


Done!


## Analyze each frame to find ants on each substrate

In [44]:

frames = 255-frames_raw
buffer = 2

print('Analyzing whole video to identify ants in each tunnel')
bkg_div_mask, ant_pix, tunnel_ant_pix, tunnel_n_ant_pix = find_ant_pix_allframes(frames, buffer, mask, 1.15, height, width, rectangle_corners)
# tunnel_n_ant_pix[tunnel_n_ant_pix>1500]=np.nan

print('save data')
sname = file[:-4] + '_TrackedData.pkl'
with open(sname, 'wb') as f:
    pickle.dump(tunnel_n_ant_pix, f)
del frames,f

print('\nDone!')


Analyzing whole video to identify ants in each tunnel
-- removing light reflections
-- taking backgrond division using 2 frames one each side of every frame
-- getting rid of small/large blobs


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater


-- finding ants in each tunnel

Done!


save data


In [300]:

FrNum = 130
buffer =1

plt.close('all')
plt.figure(figsize = (15,6))

img = frames[FrNum,:,:]
img_even = even_illumination(img, 20)

SE = np.ones((4,4)).astype(bool)
img_dil = grey_dilation(img, structure = SE)
img_clos = grey_closing(img, structure = SE)
# img_bkg = img-img_dil
img_bkg = img/img_clos
img_bkg2 = img_bkg/grey_dilation(img_bkg, structure = SE)



frames_even = np.full(frames[FrNum-buffer:FrNum+buffer+1, :,:].shape, np.nan)
for ff,fn in enumerate(range(FrNum-buffer, FrNum+buffer+1)):
    frames_even[ff]=even_illumination(frames[fn,:,:], 4)
bkg = np.mean(frames_even, axis = 0)
bkg_div =frames_even[1,:,:]/bkg
bkg_div_mask = bkg_div.copy()






glenna = frames_even[1,:,:]-even_illumination(frames[0,:,:],4)
clifton = gaussian_filter(img, 4)
talbot = img-clifton




plt.subplot(1,3,1)
plt.imshow(img, cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=255))
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(img_even, cmap = 'gray')#, norm = colors.Normalize(vmin=0, vmax=1.5))
# plt.imshow(img/clifton>1.15, 
#            cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=2))
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(img-img_even, cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=150))
plt.axis('off')

(-0.5, 1279.5, 719.5, -0.5)

In [28]:
plt.close('all')
for FrNum in range(20,23):
    img = frames[FrNum,:,:]
    img_even = even_illumination(img, 20)
    plt.figure(figsize = (10,4))
    plt.subplot(1,2,1)
    plt.imshow(img, cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=255))
    plt.subplot(1,2,2)
    plt.imshow(img_even, cmap = 'gray', norm = colors.Normalize(vmin=0, vmax=255))
    
del img, img_even, FrNum

In [23]:
pixchange = np.nansum(np.nansum(np.diff(np.divide(frames_raw[:,:,:],np.mean(frames_raw[:,:,:],axis =0)), axis = 0),axis=1),axis=1)/(720*1280)
n_bigpixchange = np.sum(np.sum(np.diff(frames_raw[:,:,300::], axis = 0)>50,axis=1),axis=1)/(720*980)
n_white = np.sum(np.sum(frames_raw[:,:,:]>230,axis=1),axis=1)
n_dark = np.sum(np.sum(frames_raw[:,:,:]<50,axis=1),axis=1)/(720*1280)
# glenna = (n_white+n_dark)/(720*1280)
glenna = (n_dark)/(720*1280)
# clifton = (n_white+n_dark)/(720*1280)
# glenna = n_white/n_dark

plt.close('all')
plt.figure()
# plt.plot(glenna)
plt.plot(pixchange, 'k')
# plt.plot(clifton)
# plt.plot(np.abs(np.diff(clifton)),'--')
# plt.axhline(y=0.28, xmin=0, xmax=1, alpha = 0.2, color = 'k')


frOI = 47
plt.figure(figsize=(10,3))
n_figs = 5
for ss,fr in enumerate( range(int(frOI-(n_figs-1)/2), int(frOI+(n_figs-1)/2+1))):
    plt.subplot(1,n_figs,ss+1)
    plt.imshow(frames_raw[fr,:,:], cmap='gray')
    plt.axis('off')
    plt.text( 20, 40, 'Fr: %s'%fr, color = [0.5, 0.5, 0.5])

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [25]:
# detect groups of ants
plt.close('all')
plt.figure(figsize=(18,6))
for FrNum in range(193,194):
    
    glenna =bkg_div_mask[FrNum,:,:]>1.15
    from scipy import ndimage
    blobs, number_of_blobs = ndimage.label(glenna)
    loc = ndimage.find_objects(blobs)


    means = np.full((number_of_blobs,2),np.nan)
    n_pts = np.full((number_of_blobs),np.nan)
    for kk in range(number_of_blobs):
        temp = np.where(blobs==kk+1)
        if len(temp[0])>6 and len(temp[0])<50:
            means[kk,:] = [np.mean(temp[1]), np.mean(temp[0])]
            n_pts[kk] = len(temp[0])
    x_dist = np.abs(means[:,0][:,None]- means[:,0][None,:])
    y_dist = np.abs(means[:,1][:,None]- means[:,1][None,:])
    total_dist = np.sqrt(np.square(x_dist)+np.square(y_dist))
    close_blobs, counts = np.unique(np.where(np.logical_and(total_dist<30,total_dist>0.001))[0], return_counts=True)
    blobs_to_remove = np.concatenate((close_blobs[counts>3], np.where(np.isnan(means[:,0]))[0]))
    blobs_to_keep = np.delete(np.arange(0,number_of_blobs),blobs_to_remove)
#     blobs_to_keep = blobs_to_keep[np.logical_not(np.isnan(means[:,0][blobs_to_keep]))]



    plt.clf()
    plt.subplot(1,2,1)
    plt.imshow(frames_raw[FrNum,:,:])
    plt.gca().axis('off')
    plt.subplot(1,2,2)
    # plt.imshow(bkg_div_mask[300,:,:]>1.15)
    # plt.imshow(np.logical_or(bkg_div_mask[300,:,:]>1.15, bkg_div_mask[300,:,:]<0.7 ))
    plt.imshow(blobs)

    for kk in range(number_of_blobs):
        if kk in blobs_to_remove:
            if not np.isnan(means[kk,0]):
                plt.text(means[kk,0], means[kk,1], '%i'%kk, Size = 6 , Color = 'w')
        elif kk in blobs_to_keep:
            plt.text(means[kk,0], means[kk,1], '%i'%kk, Size = 6 , Color = 'r')
    plt.gca().axis('off')
    plt.pause(1)




/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater


In [298]:
# display size of variables
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                  bkg_div_mask:   2.2GiB
                          mask:  42.2MiB
                           img:   7.0MiB
                          _148:   7.0MiB
                        frame2:   2.6MiB
                          _159: 900.1KiB
                          _164: 197.8KiB
                          _165: 197.8KiB
              tunnel_n_ant_pix:  15.3KiB
                          _i43:   5.0KiB


In [232]:
# generate substrate ordering
import random

for i in range(10):
    gl = [1,3,5]
    random.shuffle(gl)
    print('%i,%i,%i'%(gl[0],gl[1],gl[2]))
    cl=np.random.randint(0,2,size=3)
    print('%i,%i,%i'%(cl[0],cl[1],cl[2]))
    print('0')
    

3,5,1
1,0,1
0
1,3,5
1,1,1
0
3,1,5
0,0,0
0
5,1,3
0,1,0
0
3,1,5
0,0,0
0
1,5,3
0,1,1
0
3,1,5
0,1,0
0
1,3,5
1,0,0
0
1,5,3
1,1,1
0
5,3,1
0,0,0
0


## Past code

In [2]:
# # IMPORT INDOOR EXPERIMENT DATA
# import imageio

# pfolder = '/media/gravishlab/SeagateExpansionDrive/AntTunnelPreference/Indoor'
# folder_list = glob.glob(os.path.join(pfolder, '**Cam**'))
# folder_OI = folder_list[3]
# file_list = glob.glob(os.path.join(folder_OI, '**.jpg'))
# print(file_list[0])

# vid_length = len(file_list)
# im = imageio.imread(file_list[0])
# height = im.shape[0]
# width = im.shape[1]
# frames_raw = np.zeros((vid_length, height, width), np.uint8)
# rgb_convert = np.array([[[0.2989]],[[0.5870]],[[0.1140]]]).T
# for kk in range(vid_length):
#     im = imageio.imread(file_list[kk])
#     frames_raw[kk,:,:] = np.sum(im*rgb_convert, axis = 2)


In [ ]:
# detect yellow?
# cap = cv2.VideoCapture(file)
# cap.set(1,200)
# ret, frame2 = cap.read()
# glenna=frame2.copy()
# frame2[:,:,2]=glenna[:,:,0]
# frame2[:,:,0]=glenna[:,:,2]

# clifton = np.logical_and(
#     np.logical_and(np.abs(frame2[:,:,0]-frame2[:,:,1])<20, frame2[:,:,0]-frame2[:,:,2]>5),
#     frame2[:,:,0]>120)
# mask_all = np.sum(mask,axis=2).astype(bool)
# clifton = clifton*mask_all


# cap.release()

# plt.close('all')
# plt.figure()
# plt.subplot(1,2,1)
# plt.imshow(frame2)
# plt.subplot(1,2,2)
# plt.imshow(clifton)